In [45]:
# Import necessary initial libraries
import os
import sys
import importlib
from importlib import reload
import logging
import pandas as pd
import geopandas as gpd
import re
from datetime import timedelta
import plotly.express as px

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [7]:
# Check the current working directory
os.getcwd()

'/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms'

In [52]:
# Import necessary modules for this notebook
from  utils import (merge_geojson_grids)

In [53]:
# Reload the module to ensure the latest version is used
reload(merge_geojson_grids)

<module 'utils.merge_geojson_grids' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/utils/merge_geojson_grids.py'>

### Consolidação dos Arquivos GeoJSON das Grades Geoespaciais em um único arquivo

In [54]:
# Lista dos arquivos GeoJSON para merge (ordem importa - primeiro arquivo tem prioridade)
input_files = [
    "results/1_output_grid/partial_results/step1_name_ratio/geo1.geojson",
    "results/1_output_grid/partial_results/step1_name_ratio/geo2.geojson",
    "results/1_output_grid/partial_results/step1_name_ratio/geo3.geojson"
]

In [55]:
# output file path
output_path = "results/1_output_grid/partial_results/step1_name_ratio"
output_file_name = "step1_consolidado"

In [ ]:

# Executa o merge e salva o resultado em um arquivo GeoPackage
merge_geojson_grids.merge_geojson_to_gpkg(input_files, os.path.join(output_path, output_file_name))

Merge concluído! Arquivo salvo como GeoPackage em: results/1_output_grid/partial_results/step1_name_ratio/step1_consolidado.gpkg
CRS mantido: EPSG:4674
Total de células únicas: 8652


In [56]:
# Executa o merge e salva o resultado em um arquivo geojson
merge_geojson_grids.merge_geojson_files(input_files, os.path.join(output_path, output_file_name))

Merge concluído! Arquivo salvo em: results/1_output_grid/partial_results/step1_name_ratio/step1_consolidado.geojson


### 3. Métricas de Tempo

In [31]:
# Leitura do Arquivo Excel
file_name = [
    "results/1_output_grid/time_metrics/time_log_OHSOME_requests.xlsx"
]

In [32]:
# Leitura do Arquivo Excel
df_time = pd.read_excel(file_name[0], sheet_name='step1')

In [33]:
# Exibe as primeiras linhas para validar
display(df_time.head(10))

,Lote 1: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...,Unnamed: 2,Unnamed: 3
0,Lote 1: 25%|██▌ | 5/20 [06:16<08:19,33.27s/it]Ainda trabalhando...,NaN,NaN
1,Lote 1: 50%|█████ | 10/20 [13:24<06:19,37.98s/it]Ainda trabalhando...,NaN,NaN
2,Ainda trabalhando...,NaN,NaN,NaN
3,Lote 1: 75%|███████▌ | 15/20 [21:42<03:43,44.72s/it]Ainda trabalhando...,NaN,NaN
4,Ainda trabalhando...,NaN,NaN,NaN
5,Lote 1: 100%|██████████| 20/20 [30:34<00:00,91.71s/it],NaN,NaN
6,[SALVO] /content/drive/MyDrive/UFPR_PPGCG_DSc/...,NaN,NaN,NaN
7,[CONSOLIDADO] resultado_consolidado.geojson at...,NaN,NaN,NaN
8,Tempo do lote 1: 30.0 min 34.31 seg,NaN,NaN,NaN
9,Lote 2: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...,NaN,NaN


In [34]:
# Exibe as últimas linhas para validar
display(df_time.tail(10))

,Lote 1: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...,Unnamed: 2,Unnamed: 3
4979,[CONSOLIDADO] resultado_consolidado.geojson at...,NaN,NaN,NaN
4980,Tempo do lote 432: 23.0 min 38.74 seg,NaN,NaN,NaN
4981,Lote 433: 0%| | 0/12 [00:00<?,?it/s]Ainda trabalhando...,NaN,NaN
4982,Lote 433: 42%|████▏ | 5/12 [06:00<04:10,35.73s/it]Ainda trabalhando...,NaN,NaN
4983,Lote 433: 83%|████████▎ | 10/12 [12:06<01:09,34.71s/it]Ainda trabalhando...,NaN,NaN
4984,Lote 433: 100%|██████████| 12/12 [15:57<00:00,79.80s/it],NaN,NaN
4985,[SALVO] /content/drive/MyDrive/UFPR_PPGCG_DSc/...,NaN,NaN,NaN
4986,[CONSOLIDADO] resultado_consolidado.geojson at...,NaN,NaN,NaN
4987,Tempo do lote 433: 15.0 min 58.39 seg,NaN,NaN,NaN
4988,Processamento completo de todos os lotes.,NaN,NaN,NaN


In [38]:
# Função para extrair tempos dos lotes
def extrair_tempos(df):
    resultados = []

    for line in df.iloc[:, 0].dropna():
        match = re.search(r'Tempo do lote (\d+): (\d+\.?\d*) min (\d+\.?\d*) seg', line)
        if match:
            lote = int(match.group(1))
            minutos = float(match.group(2))
            segundos = float(match.group(3))
            total_segundos = int(minutos * 60 + segundos)
            resultados.append({
                'Lote': lote,
                'Minutos': minutos,
                'Segundos': segundos,
                'Total (s)': total_segundos,
                'Total (min)': total_segundos / 60,
                'Total (h)': total_segundos / 3600,
                'Total (dias)': total_segundos / (3600 * 24),
                'Duração (timedelta)': timedelta(seconds=total_segundos)
            })

    return pd.DataFrame(resultados)

In [42]:
# Aplicar a função para extrair os tempos
df_tempos = extrair_tempos(df_time)
df_tempos

,Lote,Minutos,Segundos,Total (s),Total (min),Total (h),Total (dias),Duração (timedelta)
0,1,30.0,34.31,1834,30.566667,0.509444,0.021227,0 days 00:30:34
1,2,31.0,23.44,1883,31.383333,0.523056,0.021794,0 days 00:31:23
2,3,27.0,41.91,1661,27.683333,0.461389,0.019225,0 days 00:27:41
3,4,28.0,20.47,1700,28.333333,0.472222,0.019676,0 days 00:28:20
4,5,25.0,27.84,1527,25.450000,0.424167,0.017674,0 days 00:25:27
...,...,...,...,...,...,...,...,...
438,429,30.0,11.31,1811,30.183333,0.503056,0.020961,0 days 00:30:11
439,430,23.0,43.62,1423,23.716667,0.395278,0.016470,0 days 00:23:43
440,431,24.0,42.73,1482,24.700000,0.411667,0.017153,0 days 00:24:42
441,432,23.0,38.74,1418,23.633333,0.393889,0.016412,0 days 00:23:38


In [43]:
# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()
print(f"Tempo Total (todos os lotes): {total_horas :.2f} horas")
print(f"Tempo Total (todos os lotes): {total_dias :.2f} dias")

Tempo Total (todos os lotes): 255.97 horas
Tempo Total (todos os lotes): 10.67 dias


In [61]:
# Gráfico de barras para visualizar os tempos de processamento
fig = px.bar(
    df_tempos,
    x='Lote',
    y='Total (min)',
    hover_data={
        'Minutos': ':.1f',
        'Segundos': ':.1f',
        'Total (h)': ':.3f'
    },
    labels={'Total (min)': 'Duração (minutos)'},
    title='Tempo de Processamento por Lote (20 células) - Step 1 (name_ratio)',
)

# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()

# Adiciona anotação com a duração total
fig.add_annotation(
    xref="paper", 
    yref="paper",
    x=0.5, 
    y=-0.35,  # Ajuste a posição vertical aqui
    text=f"Duração Total: <br>({total_horas:.2f} horas / {total_dias:.2f} dias)",
    showarrow=False,
    font=dict(size=12),
    align="center"
)

# Ajustes estéticos
fig.update_traces(marker_color='royalblue')
fig.update_layout(
    title_x=0.5,
    xaxis_title='Lote',
    yaxis_title='Duração (minutos)',
    hoverlabel=dict(bgcolor="white", font_size=13),
    margin=dict(b=100)  # Aumenta a margem inferior para caber a anotação
)

# Salva o gráfico em formato HTML
fig.write_html("step1_tempo_processamento.html")

fig.show()